In [40]:
city = 'Bozeman'
zone_field = 'ZONING2'
source_parcels = r'shp\Gallatin_GDB\Gallatin_Parcels.gdb\OwnerParcel'
source_zoning = r'shp\zoning.gdb\bozeman'

import os
import shutil
gdb = os.path.join('shp', city + '.gdb')
spatial_join_layer = "OwnerParcel_SpatialJoin_" + city

In [27]:
arcpy.conversion.FeatureClassToFeatureClass(
    source_parcels,
    gdb,
    "OwnerParcel"
)

<Result 'shp\\bozeman.gdb\\OwnerParcel'>

In [28]:
arcpy.conversion.FeatureClassToFeatureClass(
    source_zoning,
    gdb,
    "Zoning"
)

<Result 'shp\\bozeman.gdb\\Zoning'>

In [29]:
arcpy.analysis.SpatialJoin(
    "OwnerParcel", # flathead county parcels
    "Zoning", #kalispell zoning
    os.path.join(gdb, spatial_join_layer), 
    "JOIN_ONE_TO_ONE", 
    "KEEP_COMMON", 
    None, 
    "HAVE_THEIR_CENTER_IN")

<Result 'shp\\bozeman.gdb\\OwnerParcel_SpatialJoin_Bozeman'>

In [30]:
arcpy.management.SelectLayerByAttribute(
    spatial_join_layer, 
    "NEW_SELECTION", 
    zone_field + """
     IN ('R-S', 'R-1', 'R-2', 'R-3', 
     'R-4', 'R-5', 'R-O', 'RMH') 
    AND PARCELID IS NOT NULL 
    AND PropType NOT IN (
        '',
        'VR - Vacant Land Rural', 
        'CA - Centrally Assessed', 
        'VU - Vacant Land Urban',
        'FARM_U - Farmstead - Urban', 
        'NVS - Non-Valued with Specials', 
        'RV_PARK - RV Park', 
        'GRAVEL - Gravel Pit', 
        'GOLF - Golf Course',
        'EP_PART - Partial Exempt Property', 
        'CN - Centrally Assessed Non-Valued Property', 
        'NV - Non-Valued Property', 'FARM_R - Farmstead - Rural', 
        'VAC_U - Vacant Land - Urban', 
        'EP - Exempt Property', 'VAC_R - Vacant Land - Rural'
    )
    """, 
    "NON_INVERT"
)

id,value
0,a Layer object
1,11103


In [31]:
arcpy.conversion.FeatureClassToFeatureClass(
    spatial_join_layer, 
    gdb, 
    "residential_parcels"
)
# then rename to residential_parcels

<Result 'shp\\bozeman.gdb\\residential_parcels'>

In [41]:
arcpy.management.CalculateGeometryAttributes(
    "residential_parcels", 
    "lot_size AREA", 
    '', 
    "SQUARE_FEET_US", 
    None, 
    "SAME_AS_INPUT"
)

<Result 'residential_parcels'>

In [33]:
arcpy.management.Project(
    "residential_parcels", 
    os.path.join(gdb, 'parcels_ft'), 
    'PROJCS["NAD_1983_StatePlane_Montana_FIPS_2500_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",1968500.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-109.5],PARAMETER["Standard_Parallel_1",45.0],PARAMETER["Standard_Parallel_2",49.0],PARAMETER["Latitude_Of_Origin",44.25],UNIT["Foot_US",0.3048006096012192]]', "'WGS_1984_(ITRF08)_To_NAD_1983_2011 + WGS_1984_(ITRF00)_To_NAD_1983'", 'PROJCS["NAD_1983_2011_StatePlane_Montana_FIPS_2500",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",600000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-109.5],PARAMETER["Standard_Parallel_1",45.0],PARAMETER["Standard_Parallel_2",49.0],PARAMETER["Latitude_Of_Origin",44.25],UNIT["Meter",1.0]]', 
    "NO_PRESERVE_SHAPE", 
    None, 
    "NO_VERTICAL"
)

<Result 'shp\\bozeman.gdb\\parcels_ft'>

In [34]:
arcpy.management.MinimumBoundingGeometry(
    os.path.join(gdb, 'parcels_ft'), 
    os.path.join(gdb, 'parcels_mbb'), 
    "RECTANGLE_BY_WIDTH", 
    "NONE", 
    None, 
    "MBG_FIELDS"
)

<Result 'shp\\bozeman.gdb\\parcels_mbb'>

In [35]:
arcpy.management.JoinField(
    "residential_parcels", 
    'PARCELID', 
    os.path.join(gdb, 'parcels_mbb'), 
    'PARCELID', 
    'MBG_Width;MBG_Length'
)

<Result 'residential_parcels'>

In [36]:
arcpy.management.CalculateField(
    "residential_parcels", 
    "city", 
    '"' + city + '"'
)

<Result 'residential_parcels'>

In [37]:
arcpy.management.CalculateField(
    "residential_parcels", 
    "zoning", 
    "!" + zone_field + "!", 
    "PYTHON3", 
    None, 
    "TEXT"
)

<Result 'residential_parcels'>

In [5]:
arcpy.management.CalculateField(
    "residential_parcels", 
    "lot_size", 
    "round(!lot_size!)", 
    "PYTHON3", 
    None, 
    "SHORT"
)

<Result 'residential_parcels'>

In [38]:
arcpy.management.CalculateField(
    "residential_parcels", 
    "lot_width", 
    "!MBG_Width!", 
    "PYTHON3", 
    None, 
    "SHORT"
)

<Result 'residential_parcels'>

In [4]:
codeblock = """
def eval_zoning(zone):
    if zone in ['R-S', 'R-1', 'RMH']:
        return 1
    elif zone == 'R-2':
        return 2
    elif zone == 'R-3':
        return 3
    elif zone in ['R-4', 'R-5', 'R-O']:
        return 5
    else:
        return 0
"""

arcpy.management.CalculateField(
    "residential_parcels", 
    "zoning_allows", 
    "eval_zoning(!zoning!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'residential_parcels'>

In [42]:
codeblock = """
def eval_lot_size(zone, sf):
    if zone in ['R-S', 'R-1']:
        if sf >= 4000:
            return 1
        else:
            return 0
    elif zone in ['R-2']:
        if sf >= 5000:
            return 2
        elif sf >= 4000:
            return 1
        else:
            return 0
    elif zone in ['R-3']:
        if sf >= 9000:
            return 3
        if sf >= 5000:
            return 2
        if sf >= 4000:
            return 1
        else:
            return 0
    elif zone in ['R-4', 'R-O']:
        if sf >= 12000:
            return 4
        if sf >= 9000:
            return 3
        if sf >= 5000:
            return 2
        if sf >= 4000:
            return 1
        else:
            return 0
    elif zone in ['R-5']:
        if sf >= 5000:
            return 5
        if sf >= 3000:
            return 1
        else:
            return 0
    else:
        return 0
"""

arcpy.management.CalculateField(
    "residential_parcels", 
    "lot_size_allows", 
    "eval_lot_size(!zoning!, !lot_size!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'residential_parcels'>

In [44]:
codeblock = """
def eval_lot_width(zone, w):
    if zone in ['R-S', 'R-1']:
        if w >= 50:
            return 1
        else:
            return 0
    elif zone in ['R-2']:
        if w >= 60:
            return 2
        elif w >= 50:
            return 1
        else:
            return 0
    elif zone in ['R-3']:
        if w >= 60:
            return 3
        if w >= 50:
            return 1
        else:
            return 0
    elif zone in ['R-4', 'R-O']:
        if w >= 60:
            return 4
        if w >= 50:
            return 2
        else:
            return 0
    elif zone in ['R-5']:
        if w >= 50:
            return 5
        if w >= 35:
            return 1
        else:
            return 0
    else:
        return 0
"""

arcpy.management.CalculateField(
    "residential_parcels", 
    "lot_width_allows", 
    "eval_lot_width(!zoning!, !lot_width!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'residential_parcels'>

In [9]:
codeblock = """
def units(zone, size, width):
    return max(1, min(zone, size, width))
"""
arcpy.management.CalculateField(
    "residential_parcels", 
    "units_allowed", 
    "units(!zoning_allows!, !lot_size_allows!, !lot_width_allows!)",
    "PYTHON3",
    codeblock,
    "SHORT"
)

<Result 'residential_parcels'>

In [41]:
codeblock = """
def categorize(zone):
    if zone in ['R-2', 'R-3', 'R-4', 'R-5', 'R-O']:
        return "Penalized Multifamily"
    elif zone in ['R-1', 'R-S', 'RMH']:
        return "Single-Family"
"""
arcpy.management.CalculateField(
    "residential_parcels", 
    "category", 
    "categorize(!zoning!)",
    "PYTHON3",
    codeblock,
    "TEXT"
)

<Result 'residential_parcels'>

In [42]:
arcpy.conversion.FeatureClassToFeatureClass(
    "residential_parcels", 
    gdb, 
    city.lower() + "_residential_parcels", 
    '', 
    'city "city" true true false 512 Text 0 0,First,#,residential_parcels,city,0,512;zoning "zoning" true true false 512 Text 0 0,First,#,residential_parcels,zoning,0,512;lot_size "lot_size" true true false 8 Double 0 0,First,#,residential_parcels,lot_size,-1,-1;lot_width "lot_width" true true false 2 Short 0 0,First,#,residential_parcels,lot_width,-1,-1;zoning_allows "zoning_allows" true true false 2 Short 0 0,First,#,residential_parcels,zoning_allows,-1,-1;lot_size_allows "lot_size_allows" true true false 2 Short 0 0,First,#,residential_parcels,lot_size_allows,-1,-1;lot_width_allows "lot_width_allows" true true false 2 Short 0 0,First,#,residential_parcels,lot_width_allows,-1,-1;units_allowed "units_allowed" true true false 2 Short 0 0,First,#,residential_parcels,units_allowed,-1,-1;category "category" true true false 512 Text 0 0,First,#,residential_parcels,category,0,512', 
    ''
)

<Result 'shp\\Bozeman.gdb\\bozeman_residential_parcels'>